# CNN - Convolutional Neural Networks.....Tutorial 2

#### 請搭配教學影片服用
#### Tutorial video: 
- CNN概念 https://www.youtube.com/watch?v=tjcgL5RIdTM
- Tensorflow實現CNN-1 https://www.youtube.com/watch?v=tjcgL5RIdTM
- Tensorflow實現CNN-2 https://www.youtube.com/watch?v=pjjH2dGGwwY

In [1]:
#import tensorflow模組
import tensorflow as tf
#import mnist data 手寫數字資料集
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
#重要!!!!!
#清理顯卡內存
#防止顯卡內存不足報錯&提高速度
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)  
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))  

In [3]:
#讀取mnist data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
#定義compute_accuracy準確率功能___validation data
def compute_accuracy(v_xs,v_ys):
    global prediction #把prediction設成全域變量
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1}) #xs輸入到prediction生成預測值
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1)) #計算預測正確次數
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #計算準確率
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

In [6]:
#定義權重函數(輸入shape)
def weight_variable(shape):
    inital = tf.truncated_normal(shape, stddev=0.1)
    #tf.truncated_normal()函式是一種“截斷”方式生成正太分佈隨機值
    #“截斷”意思指生成的隨機數值與均值的差不能大於兩倍中誤差，否則會重新生成
    #
    #參數說明：
        #shape：表示生成隨機數的維度
        #mean：正太分佈的均值，預設為0
        #stddev：正太分佈的標準差
        #dtype：生成正太分佈資料的型別
        #seed：一個整數，當設定之後，每次生成的隨機數都一樣
        #name：正太分佈的名字
    return tf.Variable(inital)

In [7]:
#定義偏差函數(輸入shape)
def bias_variable(shape):
    inital = tf.constant(0.1,shape=shape) #生成常數0.1
    return tf.Variable(inital)

In [8]:
#定義卷積層
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1],padding='SAME') 
    #tf.nn.conv2d -> tensorflow卷積層，參數(x=輸入sample/影像size, W=卷積層權重, 
    #strides=kernel掃描影像間距(unit=px)= [1是Tensorflow規定,x方向跨度,y方向跨度,1是Tensorflow規定]) *一步掃描一次
    #strides=[1,x_movement,y_movement,1]
    #Must have strides[0] = strides[3] = 1
    #Padding參數:
    #-'SAME':kernel掃描part of patch outside of this image, it will fill the outer area as 0, 掃描完後組合起來的影像Size=原圖
    #-'VALID':kernel掃描範圍always in image,掃描完後組合起來的影像會比原圖小 

In [9]:
#定義pooling 把影像組合起來囉
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME') # <--這邊strides跨度為2 *兩步掃描一次
#tf.nn.max_pool -> max pooling卷積層，參數(x=輸入sample/影像size, 不用輸入權重,
#Must have strides[0] = strides[3] = 1
#掃描影像間距，透過把x_movement,y_movement增加，壓縮影像。 -> strides=[1,2,2,1]
#ksize = kernel shape

In [10]:
tf.reset_default_graph()
#輸入data xs->placeholder, float32型態,None不規定sample數量,784=28*28輸入影像像素
xs = tf.placeholder(tf.float32,[None,784])
#輸入data ys->placeholder, float32型態,None不規定sample數量,10=對應x的y值為，維度為10的one-hot encoding
ys = tf.placeholder(tf.float32,[None,10]) 

keep_prob = tf.placeholder(tf.float32)
#改變輸入影像型式
x_image = tf.reshape(xs, [-1, 28, 28, 1]) #tf.reshape(傳入資料xs, [-1"ignore維度", 28, 28"影像size 28*28", 1"顏色channel"])
#print(x_image.shape) #[n_sample, 28, 28, 1]


In [11]:
## conv1 layer ##
W_conv1 = weight_variable([5, 5, 1, 32]) #[5, 5"kernel size 5*5", 1"image的厚度", 32"image的高度"]
                                         #kernel 5x5, in size 1, out size 32
b_conv1 = bias_variable([32]) #out size 32
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) #hidden conv layer 1 #y=wx+b #用x_image輸入卷積層, 使用relu作激勵函數
#h_conv1 output size=28*28*32 #32="image的高度"
h_pool1 = max_pool_2x2(h_conv1) #把卷積層資料輸入到max pooling
#h_pool1 output size=14*14*32 #由於max_pool定義strides為跨2步組合影像，所以2D size減半

In [12]:
## conv2 layer ##
W_conv2 = weight_variable([5, 5, 32, 64]) #[5, 5"kernel size 5*5", 1"image的厚度", 32"image的高度"]
                                         #kernel 5x5, in size 32, out size 64
b_conv2 = bias_variable([64]) #out size 64
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) #hidden conv layer 1 #y=wx+b #用x_image輸入卷積層, 使用relu作激勵函數
#h_conv2 output size=14*14*64              #64="image的高度"
h_pool2 = max_pool_2x2(h_conv2)            #把卷積層資料輸入到max pooling
#h_pool2 output size=7*7*64                #由於max_pool定義strides為跨2步組合影像，所以2D size減半

In [13]:
## func1 layer ##
W_fc1 = weight_variable([7*7*64, 1024]) #把image的高度增大至1024 # [n_samples, 7, 7, 64] ->> [n_samples, 7*7*64]
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64]) #把影像壓平，維度->1
# [n_samples, 7, 7, 64] ->> [n_samples, 7*7*64]
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1) #y=wx+b
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob) #dropout

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
## func2 layer ##
W_fc2 = weight_variable([1024, 10]) #把image的高度增大至1024 # [n_samples, 7, 7, 64] ->> [n_samples, 7*7*64]
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2) #y=wx+b #以softmax計算結果概率 #0-9手寫圖片

In [15]:
#Loss : the error between prediction and real data
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction),axis=[1]))
#reduction_indice=[1]在新版本Tensorflow已被取代為axis=[1]

#Optimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy) #優化器 #優化函數:梯度下降 #1e-4=0.0001為Learning rate 

In [16]:
sess = tf.Session()
init = tf.global_variables_initializer() #important step
sess.run(init)

In [17]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
    if i % 50 == 0:
        print(compute_accuracy(
            mnist.test.images[:1000], mnist.test.labels[:1000]))

0.068
0.766
0.858
0.88
0.918
0.922
0.931
0.93
0.937
0.945
0.948
0.953
0.954
0.956
0.967
0.965
0.96
0.965
0.968
0.968
